In [75]:
import tensorflow as tf
import keras
import numpy as np
import os
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
import seaborn as sns

import scipy.stats as sc_stats

import random

from sklearn.model_selection import train_test_split
onehot_encoder=OneHotEncoder(sparse=False)

In [77]:
# parameters
time_steps = 1
inputs = 5
outputs = 2

In [78]:
def add_releveant_features(task_df):


    task_df['PrevOutcome']=task_df['Outcome'].shift(1)
    task_df.loc[1,'PrevOutcome']= 0

    task_df['PrevChoice']=task_df['Choice'].shift(1)
    task_df.loc[1,'PrevChoice']= 0

    task_df['PrevSafe']=task_df['Safe'].shift(1)
    task_df.loc[1,'PrevSafe']= 0

    task_df['PrevBigRisky']=task_df['BigRisky'].shift(1)
    task_df.loc[1,'PrevBigRisky']= 0

    task_df['PrevSmallRisky']=task_df['SmallRisky'].shift(1)
    task_df.loc[1,'PrevSmallRisky']= 0
    
#     task_df['PrevRT']=task_df['RT'].shift(1)
#     task_df.loc[1,'PrevRT']= 0
    
    
    
    return task_df

In [79]:
def chunk_split_data(data,start_chunk,end_chunk):
    
    a=[k for k in range(start_chunk,end_chunk)]
    out=[]

    for d in range(0,data.shape[0],20):

        c= [c+d for c in a]
        out = out+c

    while out[-1]>=data.shape[0]-1:
        out.pop()
#     return out
    return data[out]

In [80]:
def train_RNN(neurons,train_X,train_y,test_X,test_y,val_X,val_y): 
    reset_graph()

    learning_rate = 0.001
    epochs = 50000
    batch_size = int(train_X.shape[0]/2)
    # batch_size = 100
    length = train_X.shape[0]
    display = 100
    neurons = neurons

    num_batches = 100
    seq_len = 10

    percent_above_PT = 1

    train_threshold = 1.5#PT_R2 + percent_above_PT


    save_step = 100


    best_loss_val = np.infty
    checks_since_last_progress = 0
    max_checks_without_progress = 1000


    # clear graph (if any) before running
    tf.reset_default_graph()

    X = tf.placeholder(tf.float32, [None, time_steps, inputs])

    y = tf.placeholder(tf.float32, [None, outputs])

    # LSTM Cell
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=neurons, activation=tf.nn.relu)
    cell_outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

    # pass into Dense layer
    stacked_outputs = tf.reshape(cell_outputs, [-1, neurons])
    out = tf.layers.dense(inputs=stacked_outputs, units=outputs)

    probability = tf.nn.softmax(out)

    # squared error loss or cost function for linear regression
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=y, logits=out))

    # optimizer to minimize cost
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    saver = tf.train.Saver()
    
    accuracy = tf.metrics.accuracy(labels =  tf.argmax(y, 1),
                          predictions = tf.argmax(out, 1),
                          name = "accuracy")
    precision = tf.metrics.precision(labels=tf.argmax(y, 1),
                                 predictions=tf.argmax(out, 1),
                                 name="precision")
    recall = tf.metrics.recall(labels=tf.argmax(y, 1),
                           predictions=tf.argmax(out, 1),
                           name="recall")
    f1 = 2 * accuracy[1] * recall[1] / ( precision[1] + recall[1] )

    acc_up,acc_val = accuracy
    auc = tf.metrics.auc(labels=tf.argmax(y, 1),
                           predictions=tf.argmax(out, 1),
                           name="auc")
    
    valid_store = []
    
    with tf.Session() as sess:
        #######################
#         saver.restore(sess, "./checkpts/Original_RNN_LSTM_8features_v2.ckpt")
#         saver.restore(sess, "./checkpts/OriginalDATA_RNN_LSTM_8features.ckpt")
        
        if pretraining == True:

            saver.restore(sess, "./checkpts/Original_v2_DATA_RNN_LSTM_8features.ckpt")

        #######################
        
        # initialize all variables
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()

        # Train the model
        for steps in range(epochs):
            mini_batch = zip(range(0, length, batch_size),
                       range(batch_size, length+1, batch_size))

            # train data in mini-batches
            for (start, end) in mini_batch:
    #             print(start,end)
                sess.run(training_op, feed_dict = {X: train_X[start:end,:,:],
                                                   y: train_y[start:end,:]}) 

            ## train data in batches of length subsequence

    #         for k in range(num_batches):
    #             X_seq, y_seq = random_subsequence(train_X,train_y,seq_len)

    #             sess.run(training_op, feed_dict = {X:X_seq,y:y_seq}) 
            loss_fn = loss.eval(feed_dict = {X: train_X, y: train_y})
            loss_val = loss.eval(feed_dict={X: val_X, y: val_y})


            # print training performance 
            if (steps+1) % display == 0:
                # evaluate loss function on training set


                loss_fn = loss.eval(feed_dict = {X: train_X, y: train_y})
                print('Step: {}  \tTraining loss: {}'.format((steps+1), loss_fn))

                acc_train = acc_val.eval(feed_dict={X: train_X, y: train_y})
                print('Step: {}  \tTraining accuracy: {}'.format((steps+1), acc_train))


                acc_test = acc_val.eval(feed_dict={X: test_X, y: test_y})
    #             print('Step: {}  \tTest accuracy: {}'.format((steps+1), acc_test))

                loss_test = loss.eval(feed_dict={X: test_X, y: test_y})
    #             print('Step: {}  \tTest loss: {}'.format((steps+1), loss_test))

                accu_val = acc_val.eval(feed_dict={X: val_X, y: val_y})

                loss_val = loss.eval(feed_dict={X: val_X, y: val_y})
                print('Step: {}  \tValid loss: {}'.format((steps+1), loss_val))

                valid_store.append(loss_val)







            if (1 + loss_fn/np.log(0.5)) > train_threshold:
                    print("Threshold achieved, quit training")
                    break


            if loss_val < best_loss_val:

                        best_loss_val = loss_val
                        checks_since_last_progress = 0
            else:
                            checks_since_last_progress += 1


            # EARLY STOPPING
            if checks_since_last_progress > max_checks_without_progress:
                print("Early stopping!")
                break


            if (steps+1) % save_step ==0:
                                save_path = saver.save(sess, "./checkpts/Later_RNN_LSTM_8features.ckpt")

#                 save_path = saver.save(sess, "./checkpts/RNN_Internet_LSTM_model_5features.ckpt")





    #     evaluate model accuracy
        acc, prec, recall, f1, AUC = sess.run([accuracy, precision, recall, f1,auc],
                                         feed_dict = {X: train_X, y: train_y})
        prob_train = probability.eval(feed_dict = {X: train_X, y: train_y})
        prob_test = probability.eval(feed_dict = {X: test_X, y: test_y})
        prob_valid = probability.eval(feed_dict = {X: val_X, y: val_y})



        print('\nEvaluation  on training set')
        print('Accuracy:', acc[1])
        print('Precision:', prec[1])
        print('Recall:', recall[1])
        print('F1 score:', f1)
        print('AUC:', AUC[1])
        
      
    
    
#         save_path = saver.save(sess, "./checkpts/Original_v2_DATA_RNN_LSTM_8features.ckpt")
#         save_path = saver.save(sess, "./checkpts/Later_v2_DATA_RNN_LSTM_8features.ckpt")

        
#         save_path = saver.save(sess, "./checkpts/OriginalDATA_RNN_LSTM_8features.ckpt")
#         save_path = saver.save(sess, "./checkpts/LaterDATA_RNN_LSTM_8features.ckpt")


#         save_path = saver.save(sess, "./checkpts/Original_RNN_LSTM_8features.ckpt")

#         save_path = saver.save(sess, "./checkpts/Later_RNN_LSTM_8features.ckpt")




    ## APP DATA
#         save_path = saver.save(sess, "./checkpts/Original_v2_APPDATA_RNN_LSTM_8features.ckpt")
        save_path = saver.save(sess, "./checkpts/Later_v2_APPDATA_RNN_LSTM_8features.ckpt")


    metric_out_df= pd.DataFrame(np.array([acc[1],prec[1],recall[1],f1,AUC[1],loss_fn,accu_val,best_loss_val,acc_test,loss_test,neurons,learning_rate,epochs,steps]).reshape(-1,14),columns =["accuracy","precision","recall","f1_score","auc","loss","accuracy_val","loss_val","accuracy_test","loss_test","neurons","learning_rate","n_epochs","steps"])
    return metric_out_df, prob_train, prob_test, prob_valid
    


    
    
    
    
def random_subsequence(X,y,seq_len):
    rnd  = random.randint(0,len(X)-seq_len)
    X_seq, y_seq = X[rnd:rnd+seq_len,:], y[rnd:rnd+seq_len,:]
    return X_seq, y_seq

    print(y_seq.shape)

## Odd plays train, even plays test and valid


In [81]:
dir_list = os.listdir("/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/")
dir_path ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"

subj_files_list =[]; ## list of subject_files fullfilling a criteria

dir_files = [i for i in os.listdir(dir_path) if i.startswith('sub')]

for subj_file_path in dir_files:

    file_path  ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"+ subj_file_path
    mypath =file_path
    
    play_names = [i for i in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,i)) and i.startswith('app')]   
    
    if len(play_names) >= 50: ## criteria
        subj_files_list.append(subj_file_path)
        
    

In [82]:
len(subj_files_list)

37

In [46]:
num_shuffles=1
for num, subj_file_path in enumerate(subj_files_list):
    print(num)
# for subj_file_path in [subj_files_list[0]]:
    
#     train_data,test_data, val_data = np.empty((0,task_df.columns.shape[0])),  np.empty((0,task_df.columns.shape[0])), np.empty((0,task_df.columns.shape[0]))
    train_data,test_data, val_data = np.empty((0,15)),  np.empty((0,15)), np.empty((0,15))

    file_path  ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"+ subj_file_path
    mypath =file_path
        
    comp_task_train_df = pd.DataFrame()

    play_names = [i for i in os.listdir(mypath) if os.path.isfile(os.path.join(mypath,i)) and i.startswith('app')]    

    for randomization_counter in range(0,num_shuffles):
            randomized_play_names= random.sample(play_names,len(play_names))
            
            for play_num, play_name in enumerate(randomized_play_names):
#         for play_num,play_name in enumerate(play_names):

                file_name = file_path + "/" + play_name
                task_df = pd.read_csv(file_name)
                task_df = add_releveant_features(task_df)

                if np.mod(play_num,2)==0: ## odd trials
                    train_data = np.append(train_data,task_df[task_df.TrialNum>1].values, axis=0)

                else:
                    test_data =  np.append(test_data, task_df[task_df.TrialNum>1].values[0:16], axis=0)
                    val_data =  np.append(val_data, task_df[task_df.TrialNum>1].values[16:], axis=0)


    train_data_df= pd.DataFrame(train_data,columns=task_df.columns)
    val_data_df = pd.DataFrame(test_data,columns=task_df.columns)
    test_data_df= pd.DataFrame(val_data,columns=task_df.columns)

#     file_path = file_path + "/OddEvenPlays/"
    file_path = file_path + "/OddEvenPlays/RandomizedPlays1"

#     os.mkdir(file_path)
    train_data_df.to_csv(file_path+"/train_data.csv")
    test_data_df.to_csv(file_path+"/test_data.csv")
    val_data_df.to_csv(file_path+"/val_data.csv")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


In [85]:
def data_split_odd_even(train_data_df,test_data_df,val_data_df):

#     train_len = 29
#     test_len = 14
#     val_len = 15

    ##----------------- UNCOMMENT BELOW
    
    if hist_flag==0: ## CURR OPTIONS ONLY
        
    
        train_X = train_data_df[['Safe','BigRisky','SmallRisky']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)

    elif hist_flag==1: ## CURR OPTIONS, PREV ACTIONS:
        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevChoice']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevChoice']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevChoice']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
        
        
    elif hist_flag==2: ## CURR OPTIONS, PREV ACTIONS, PREV OUTCOME
        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevChoice','PrevOutcome']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevChoice','PrevOutcome']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevChoice','PrevOutcome']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
             
        

####### Prev O + C+ R + CurrO--------------------
    elif hist_flag==3: # CURR OPTIONS, PREV ACTIONS, PREV OUTCOME, PREV OPTIONS
        
        train_X = train_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
        train_y = train_data_df[['Choice']].values.astype(np.int32)

        test_X = test_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
        test_y = test_data_df[['Choice']].values.astype(np.int32)

        val_X = val_data_df[['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
        val_y = val_data_df[['Choice']].values.astype(np.int32)
    
    
    
    
    ######## sampling 
    
    
#### - Prev RT+C+R+O + Curr O----------------------

#     train_X = task_df.loc[task_df.TrialNum>1, ['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky','PrevRT']].values
#     train_y = task_df.loc[task_df.TrialNum>1,['Choice']].values.astype(np.int32)

#     test_X = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky','PrevRT']].values
#     test_y = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Choice']].values.astype(np.int32)






    #### PRE TRAINING
#     stop = int(0.7*len(train_X))
#     print(stop)
#     train_X, test_X, val_X, train_y, test_y, val_y= train_X[:stop], train_X[stop:stop+int((len(train_X)-stop)/2)], train_X[stop+int((len(train_X)-stop)/2):],train_y[:stop], train_y[stop:stop+int((len(train_X)-stop)/2)], train_y[stop+int((len(train_X)-stop)/2):]
    
#     train_X, test_X, val_X, train_y, test_y, val_y = train_X, test_X, test_X, train_y, test_y, test_y
    ###################################################################


    print(train_X.shape)
    print(train_y.shape)
    print(val_X.shape)
    print(val_y.shape)
    print(test_X.shape)
    print(test_y.shape)

    # # center and scale
    scaler = MinMaxScaler(feature_range=(0, 1))    
    train_X = scaler.fit_transform(train_X)
    test_X = scaler.fit_transform(test_X)
    val_X = scaler.fit_transform(val_X)


    train_X = train_X[:,None,:]
    val_X = val_X[:,None,:]
    test_X = test_X[:,None,:]


    # # one-hot encode the outputs

    onehot_encoder = OneHotEncoder()
    encode_categorical = train_y.reshape(len(train_y), 1)
    encode_categorical_test = test_y.reshape(len(test_y), 1)
    encode_categorical_val = val_y.reshape(len(val_y),1)


    train_y = onehot_encoder.fit_transform(encode_categorical).toarray()
    test_y = onehot_encoder.fit_transform(encode_categorical_test).toarray()
    val_y = onehot_encoder.fit_transform(encode_categorical_val).toarray()

    
    return train_X, train_y, test_X, test_y, val_X,val_y


In [ ]:
neurons = 8
hist_flag=2



for num, subj_file_path in enumerate(subj_files_list):
    print(num)
# for subj_file_path in [subj_files_list[0]]:
    
    file_path  ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/appdata/"+ subj_file_path
                
#     file_path = file_path + "/OddEvenPlays"
    file_path = file_path + "/OddEvenPlays/RandomizedPlays1"

    train_data_df= pd.read_csv(file_path+"/train_data.csv")
    test_data_df = pd.read_csv(file_path+"/test_data.csv")
    val_data_df = pd.read_csv(file_path+"/val_data.csv")

    train_X, train_y, test_X, test_y,val_X,val_y = data_split_odd_even(train_data_df,test_data_df,val_data_df)

    pretraining = False; 
    metric_out_df, prob_train, prob_test, prob_val = train_RNN(neurons,train_X,train_y,test_X,test_y,val_X,val_y)
    
    print(metric_out_df)
    
   
 
    prob_train_df = pd.DataFrame(prob_train.reshape(-1,2),columns = {'action_0','action_1'})
    prob_test_df = pd.DataFrame(prob_test.reshape(-1,2),columns = {'action_0','action_1'})
    prob_val_df = pd.DataFrame(prob_val.reshape(-1,2),columns = {'action_0','action_1'})

    if hist_flag==0:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currO_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currO_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currO_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currO_neurons="+str(neurons)+".csv")
    
    
    elif hist_flag==1:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currOprevC_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currOprevC_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currOprevC_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currOprevC_neurons="+str(neurons)+".csv")

    elif hist_flag==2:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currOprevRC_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currOprevRC_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currOprevRC_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currOprevRC_neurons="+str(neurons)+".csv")
# ################################
    elif hist_flag==3:
        metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currprev_opts_metricsneurons="+str(neurons)+".csv")
        prob_train_df.to_csv(file_path + "/prob_train_currentprevopts_neurons="+str(neurons)+".csv")
        prob_test_df.to_csv(file_path + "/prob_test_currentprevopts_neurons="+str(neurons)+".csv")
        prob_val_df.to_csv(file_path + "/prob_val_currentprevopts_neurons="+str(neurons)+".csv")
# #############################

0
(1189, 5)
(1189, 1)
(640, 5)
(640, 1)
(520, 5)
(520, 1)


/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.35643652081489563
Step: 100  	Training accuracy: 0.8696383237838745
Step: 100  	Valid loss: 0.37299397587776184
Step: 200  	Training loss: 0.3162817060947418
Step: 200  	Training accuracy: 0.8651780486106873
Step: 200  	Valid loss: 0.35062503814697266
Step: 300  	Training loss: 0.2835092544555664
Step: 300  	Training accuracy: 0.8642772436141968
Step: 300  	Valid loss: 0.3428322374820709
Step: 400  	Training loss: 0.26558586955070496
Step: 400  	Training accuracy: 0.8671685457229614
Step: 400  	Valid loss: 0.350076287984848
Step: 500  	Training loss: 0.2614310681819916
Step: 500  	Training accuracy: 0.8701936602592468
Step: 500  	Valid loss: 0.3593922257423401
Step: 600  	Training loss: 0.2606891691684723
Step: 600  	Training accuracy: 0.871192216873169
Step: 600  	Valid loss: 0.3619684875011444
Step: 700  	Training loss: 0.26032280921936035
Step: 700  	Training accuracy: 0.8718838095664978
Step: 700  	Valid loss: 0.3620494306087494
Step: 800  	Training los

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.5127460360527039
Step: 100  	Training accuracy: 0.7593244314193726
Step: 100  	Valid loss: 0.5398445725440979
Step: 200  	Training loss: 0.48025959730148315
Step: 200  	Training accuracy: 0.7556493878364563
Step: 200  	Valid loss: 0.5049644112586975
Step: 300  	Training loss: 0.4412953853607178
Step: 300  	Training accuracy: 0.7620924115180969
Step: 300  	Valid loss: 0.462371826171875
Step: 400  	Training loss: 0.4092418849468231
Step: 400  	Training accuracy: 0.7701149582862854
Step: 400  	Valid loss: 0.4259050190448761
Step: 500  	Training loss: 0.3968777060508728
Step: 500  	Training accuracy: 0.7774128913879395
Step: 500  	Valid loss: 0.4110395908355713
Step: 600  	Training loss: 0.3942534327507019
Step: 600  	Training accuracy: 0.7829526662826538
Step: 600  	Valid loss: 0.407516211271286
Step: 700  	Training loss: 0.393618106842041
Step: 700  	Training accuracy: 0.7870044708251953
Step: 700  	Valid loss: 0.4066471755504608
Step: 800  	Training loss: 0.

...

Step: 100  	Training loss: 0.5430371761322021
Step: 100  	Training accuracy: 0.8316429853439331
Step: 100  	Valid loss: 0.5554499626159668
Step: 200  	Training loss: 0.3983464241027832
Step: 200  	Training accuracy: 0.8299526572227478
Step: 200  	Valid loss: 0.4578074514865875
Step: 300  	Training loss: 0.3713687062263489
Step: 300  	Training accuracy: 0.8334685564041138
Step: 300  	Valid loss: 0.44260886311531067
Step: 400  	Training loss: 0.3585689663887024
Step: 400  	Training accuracy: 0.8355548977851868
Step: 400  	Valid loss: 0.43044978380203247
Step: 500  	Training loss: 0.35037875175476074
Step: 500  	Training accuracy: 0.8378408551216125
Step: 500  	Valid loss: 0.42204800248146057
Step: 600  	Training loss: 0.3444254994392395
Step: 600  	Training accuracy: 0.8388345837593079
Step: 600  	Valid loss: 0.41650891304016113
Step: 700  	Training loss: 0.3404620885848999
Step: 700  	Training accuracy: 0.8395225405693054
Step: 700  	Valid loss: 0.41396573185920715
Step: 800  	Training 

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.598982572555542
Step: 100  	Training accuracy: 0.6077896356582642
Step: 100  	Valid loss: 0.6053254008293152
Step: 200  	Training loss: 0.4371595084667206
Step: 200  	Training accuracy: 0.6903167366981506
Step: 200  	Valid loss: 0.4548654854297638
Step: 300  	Training loss: 0.3552290201187134
Step: 300  	Training accuracy: 0.7602083683013916
Step: 300  	Valid loss: 0.37606143951416016
Step: 400  	Training loss: 0.31724148988723755
Step: 400  	Training accuracy: 0.7932451963424683
Step: 400  	Valid loss: 0.3377515375614166
Step: 500  	Training loss: 0.29383304715156555
Step: 500  	Training accuracy: 0.8127291202545166
Step: 500  	Valid loss: 0.3127574622631073
Step: 600  	Training loss: 0.27753663063049316
Step: 600  	Training accuracy: 0.8258722424507141
Step: 600  	Valid loss: 0.2940753400325775
Step: 700  	Training loss: 0.26576802134513855
Step: 700  	Training accuracy: 0.836059033870697
Step: 700  	Valid loss: 0.2805619239807129
Step: 800  	Training los

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

Step: 100  	Training loss: 0.4573133885860443
Step: 100  	Training accuracy: 0.8332160711288452
Step: 100  	Valid loss: 0.4707096517086029
Step: 200  	Training loss: 0.43507376313209534
Step: 200  	Training accuracy: 0.8411916494369507
Step: 200  	Valid loss: 0.4511728584766388
Step: 300  	Training loss: 0.4155980050563812
Step: 300  	Training accuracy: 0.8426460027694702
Step: 300  	Valid loss: 0.43217819929122925
